<center> <img alt=\"DAGsHub\" width=500px src=https://raw.githubusercontent.com/DAGsHub/client/master/dagshub_github.png> </center>

# SavtaDepth Colab Environment

### ***This notebook is meant to be run on Google Colab***

This notebook is a tool to setup and orchestrate (read: Run) experiments for [SavtaDepth](https://dagshub.com/OperationSavta/SavtaDepth) on Google Colab, while maintaining a relatively clean environment and using version control to promote reproducibility. It is a WIP, but following (and modifying) the cells below should let you git clone a project into colab, download the data, run it, and push it to https://DAGsHub.com which is a free platform for open source data science. 

SavtaDepth is an Open Source Data Science project. We'd love to get help from the community, so if you'd like to contribute head over to the [project page](https://dagshub.com/OperationSavta/SavtaDepth) to get started.

# General Setup
Run this before anything else

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from google.colab import auth
auth.authenticate_user()

Clone Git Repo from DAGsHub

In [ ]:
!git clone https://dagshub.com/OperationSavta/SavtaDepth.git
%cd SavtaDepth/

*Note: Currently you can't see the intermediate output so it's completely opaque and long process + you can't set up credentials for google within the shell instance*

This installs conda, creates a virtual environment and installs all relevant requirements.

In [ ]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-py37_4.8.3-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX
rm $MINICONDA_INSTALLER_SCRIPT
make env

Since locally we might not be working with a GPU, here we install the pytorch version that should use the GPU provided by Google Colab. After that we install the rest of the requirements from the requirements.txt file.

In [ ]:
!bash -c "source activate savta_depth && conda install -y pytorch torchvision cudatoolkit=10.1 -c pytorch && make load_requirements"

### Pull DVC files from our remote

In [ ]:
!bash -c "source activate savta_depth && dvc pull -r dvc-remote"



---


# Setup is done!
If you've made it here, everything is set up. You have the code and data in the file viewer to the left. You can edit the files located in SavtaDepth/src/code/ as you like. You can see the YAML defining the project pipeline in dvc.yaml. If you change dependencies or outputs (for example, add an addition code file for the training stage), make sure you edit the pipeline to reflect this.

### Once you are done with you changes run the cell below to run the pipeline end-to-end
* You can run this multiple times if you've made a change and want to test it
* If you want to run only a specific stage you can change the `dvc repro` command to any other command you like.

▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽

In [ ]:
!bash -c "source activate savta_depth && dvc repro"

---
# Commiting Your Work and Pushing Back to DAGsHub



In [ ]:
!git status

In [ ]:
# Add the files you want to commit
!git add {your files here}

Run the following 2 cells without modifications. They will prompt you for a commit message, and for credentials to push back to DAGsHub

**Commiting**

In [ ]:
os.environ['COMMIT_MESSAGE'] = input('Enter the commit message for you commit: ')
!git commit -m "${COMMIT_MESSAGE}"
os.environ['COMMIT_MESSAGE'] = ""

**Pushing to DAGsHub**

In [ ]:
# If this stage fails, make sure to remove outputs as it will show your password to whoever it is shared with.
from getpass import getpass
import os

os.environ['USER'] = input('Enter the username of your DAGsHub account: ')
os.environ['PASSWORD'] = getpass('Enter the password of your DAGsHub account: ')
os.environ['REPO_URL'] = input('Enter the url of your DAGsHub project: ').split('https://')[-1]
os.environ['DAGSHUB_AUTH'] = os.environ['USER'] + ':' + os.environ['PASSWORD']

!git push https://$DAGSHUB_AUTH@$REPO_URL.git

***NOTE: ALWAYS RUN THIS CELL AFTER THE PREVIOUS***, it will delete your DAGsHub password in case you share this notebook with someone

In [ ]:
os.environ['PASSWORD'] = os.environ['DAGSHUB_AUTH'] = ""

## Push data back to your DVC Remote
For this step you must first create a DVC remote on some cloud provider. We recommend Google Cloud Storage. If you're not sure how to set up a DVC remote, [follow these instructions](https://dagshub.com/docs/getting-started/set-up-remote-storage-for-data-and-models/#create-a-storage-bucket) (you only need to go through creating a storage bucket and adding permissions).

Add your storage bucket to DVC (replace {bucket-name} with the bucket name you chose)

In [ ]:
!bash -c "source activate savta_depth && dvc remote add my-dvc-remote gs://{bucket-name}"

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!bash -c "source activate savta_depth && dvc push -r my-dvc-remote"

# That's it, you can now create a PR on DAGsHub.